In [1]:
import sys
sys.path.insert(0, "../../price_estimator/price-estimator-rest-api/api")
sys.path.insert(0, "../../price_estimator/price-estimator-rest-api/")
sys.path.insert(0, "../../price_estimator/price-estimator-rest-api/proto")

In [2]:
from proto.realty.offer import common_pb2, RealtySchema_pb2

In [3]:
from proto.realty.prediction import price_prediction_pb2
from proto.realty import model_pb2
from proto.realty.offer.unified_offer_parts import address_part_pb2, offer_category_part_pb2, object_infos_part_pb2

In [4]:
from proto.vertis.vsml.features.realty_price_prediction_pb2 import RealtyPricePredictionFeatures, PriceStatistics

In [5]:
from base64 import b64encode, b64decode

In [6]:
from api.io_helper import protobuf_to_dict

## Создание протобафов для тестов

см `realty/price_estimator/price-estimator-rest-api/test/test_protobuf.py`

### 1. RealtySchema_pb2

делаем по подобию готового протобафа, который предварительно скачали [из снепшота](https://yt.yandex-team.ru/hahn/navigation?path=//home/verticals/realty/production/export/offers/snapshot)

In [8]:
request_path = "../../price_estimator/price-estimator-rest-api/proto.request.sell.msc.example.binary"

In [9]:
message = RealtySchema_pb2.OfferMessage()

In [10]:
with open(request_path, "rb") as f:
    data = f.read()

In [11]:
message.ParseFromString(data)

4254

In [13]:
new_message = RealtySchema_pb2.OfferMessage(
    apartmentInfo=RealtySchema_pb2.ApartmentInfoMessage(
        apartment=message.apartmentInfo.apartment,
        studio=message.apartmentInfo.studio,
        floors=message.apartmentInfo.floors,
        ceilingHeight=message.apartmentInfo.ceilingHeight,
        roomsOffered=message.apartmentInfo.roomsOffered,
        rooms=message.apartmentInfo.rooms,
        balconyInt=message.apartmentInfo.balconyInt,
        renovationInt=message.apartmentInfo.renovationInt,
        flatTypeInt=message.apartmentInfo.flatTypeInt
    ),
    offerTypeInt=message.offerTypeInt,
    categoryTypeInt=message.categoryTypeInt,
    buildingInfo=RealtySchema_pb2.BuildingInfoMessage(
        floorsTotal=message.buildingInfo.floorsTotal,
        buildingId=message.buildingInfo.buildingId,
        buildYear=message.buildingInfo.buildYear,
        buildingSeriesId=message.buildingInfo.buildingSeriesId,
        expectDemolition=message.buildingInfo.expectDemolition,
        parkingTypeInt=message.buildingInfo.parkingTypeInt,
        flatsCount=message.buildingInfo.flatsCount,
        site_id=500000, #message.buildingInfo.site_id,
#         has_elevator=??
    ),
    location=RealtySchema_pb2.LocationMessage(
        localityName="test_locality_name", #message.location.localityName,
        subjectFederationId=1, #message.location.subjectFederationId,
        regionGraphId=123456, #message.location.regionGraphId,
        geocoderAddress="test_geocoder_address", #message.location.geocoderAddress,
        geocoderPoint=RealtySchema_pb2.GeoPointMessage(
            latitude=99.000000,
            longitude=66.000000
        )
    ),
    lotInfo=RealtySchema_pb2.LotInfoMessage(
        lotArea=RealtySchema_pb2.AreaInfoMessage(
            unitInt=message.lotInfo.lotArea.unitInt,
            value=message.lotInfo.lotArea.value
        ),
        lotTypeInt=message.lotInfo.lotTypeInt
    ),
    houseInfo=RealtySchema_pb2.HouseInfoMessage(
        pmg=message.houseInfo.pmg,
        houseTypeInt=message.houseInfo.houseTypeInt,
        livingSpace=message.houseInfo.livingSpace,
        kitchenSpace=message.houseInfo.kitchenSpace
    ),
    area=message.area,
    transaction=RealtySchema_pb2.TransactionMessage(
        area=message.transaction.area
    )
)

In [14]:
new_data = new_message.SerializeToString()

In [16]:
b64encode(new_data)

b'IAEwAlJAOhJ0ZXN0X2xvY2FsaXR5X25hbWVaChUAAMZCHQAAhEKCARV0ZXN0X2dlb2NvZGVyX2FkZHJlc3OwAcDEB+ABAVoMigEJCAEVAABgQhgB2gEJCAEVAABgQhgB4gENMgEESAJQApABAMgBAeoBEXCgwh6gAQj4AQCgAhjAAuEP8gECQAD6AQISAA=='

In [18]:
# protobuf_to_dict(new_message.SerializeToString())

### 2. PricePredictionLandingRequest

делаем по подобию готового протобафа из п.1

In [19]:
message = price_prediction_pb2.PricePredictionLandingRequest()

message.apartment.apartment_area = 38.2
message.apartment.rooms_total = 1
message.apartment.general_apartment_info.ceiling_height.value = 0

message.apartment.building_info.built_year.value = 2011
message.apartment.building_info.flats_count.value = 0
message.apartment.building_info.expect_demolition.value = False
message.apartment.building_info.has_lift.value = True
message.apartment.building_info.building_id.value = 0
message.apartment.building_info.building_series.id = 0
message.apartment.building_info.building_type = 0
message.apartment.building_info.floors_total.value = 9

message.location.geocoder_coordinates.latitude = 99.0
message.location.geocoder_coordinates.longitude = 66.0


c = address_part_pb2.Address.Component()
c.region_type = model_pb2.RegionType.Value('CITY')
c.value = 'test_locality_name'
message.location.geocoder_address.component.append(c)
message.location.geocoder_address.unified_oneline = "test_geocoder_address"

message.location.subject_federation_geoid = 1

In [20]:
b64encode(message.SerializeToString())

b'CiQNzcwYQjABOhcKACoAOgIICWIDCNsPcgIIAbIBALoBAEoCEgASQDIvChV0ZXN0X2dlb2NvZGVyX2FkZHJlc3MSFgoSdGVzdF9sb2NhbGl0eV9uYW1lGAc6Cg0AAMZCFQAAhEKgAQE='

In [21]:
# protobuf_to_dict(message.SerializeToString(), protobuf_model="PriceLandingPredictionRequest")

### 3. RealtyPricePredictionFeatures

#### Полная версия

In [22]:
message = RealtyPricePredictionFeatures(
    rent_apartment=PriceStatistics(
        min=1,
        q25=2,
        median=3,
        q75=4,
        max=5,
        count=6
    ),
    rent_room=PriceStatistics(
        min=1,
        q25=2,
        median=3,
        q75=4,
        max=5,
        count=6
    ),
    sell_apartment=PriceStatistics(
        min=1,
        q25=2,
        median=3,
        q75=4,
        max=5,
        count=6
    ),
    sell_room=PriceStatistics(
        min=1,
        q25=2,
        median=3,
        q75=4,
        max=5,
        count=6
    ),
    object_type=2
)

In [24]:
new_data = message.SerializeToString()

In [25]:
b64encode(new_data)

b'ChsNAACAPxUAAABAHQAAQEAlAACAQC0AAKBAMAYSGw0AAIA/FQAAAEAdAABAQCUAAIBALQAAoEAwBhobDQAAgD8VAAAAQB0AAEBAJQAAgEAtAACgQDAGIhsNAACAPxUAAABAHQAAQEAlAACAQC0AAKBAMAYwAg=='

In [27]:
# protobuf_to_dict(message.SerializeToString(), protobuf_model="RealtyPricePredictionFeatures")

#### Сокращенная версия

In [29]:
message = RealtyPricePredictionFeatures(
    sell_apartment=PriceStatistics(
        min=9,
        q25=10,
        median=11,
        q75=12,
        max=13,
        count=14
    )
)

In [30]:
new_data = message.SerializeToString()

In [31]:
b64encode(new_data)

b'GhsNAAAQQRUAACBBHQAAMEElAABAQS0AAFBBMA4='

In [33]:
# protobuf_to_dict(message.SerializeToString(), protobuf_model="RealtyPricePredictionFeatures")